In [48]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [49]:
data = pd.read_csv("../Dataset/train.csv")
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [51]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler, PowerTransformer, OrdinalEncoder
from sklearn.model_selection import train_test_split

In [52]:
X_train, X_val, y_train, y_val = train_test_split(data.drop("Survived", axis=1), data["Survived"], test_size=0.2, random_state=0)

In [53]:
X_train

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
140,141,3,"Boulos, Mrs. Joseph (Sultana)",female,NaN,0,2,2678,15.2458,NaN,C
439,440,2,"Kvillner, Mr. Johan Henrik Johannesson",male,31.0,0,0,C.A. 18723,10.5000,NaN,S
817,818,2,"Mallet, Mr. Albert",male,31.0,1,1,S.C./PARIS 2079,37.0042,NaN,C
378,379,3,"Betros, Mr. Tannous",male,20.0,0,0,2648,4.0125,NaN,C
491,492,3,"Windelov, Mr. Einar",male,21.0,0,0,SOTON/OQ 3101317,7.2500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
835,836,1,"Compton, Miss. Sara Rebecca",female,39.0,1,1,PC 17756,83.1583,E49,C
192,193,3,"Andersen-Jensen, Miss. Carla Christine Nielsine",female,19.0,1,0,350046,7.8542,NaN,S
629,630,3,"O'Connell, Mr. Patrick D",male,NaN,0,0,334912,7.7333,NaN,Q
559,560,3,"de Messemaeker, Mrs. Guillaume Joseph (Emma)",female,36.0,1,0,345572,17.4000,NaN,S


In [54]:
DropAndImpute = ColumnTransformer(
    transformers=[
        ("AgeImpute", SimpleImputer(strategy='mean', copy=False), [4]),
        ("EmbarkedImpute", SimpleImputer(strategy='most_frequent', copy=False), [10]),
        ("CabinImpute", SimpleImputer(strategy='constant', fill_value=0 ,add_indicator=True),[9]), 
    ], remainder="passthrough"
)

In [55]:
DropAndImpute.set_output(transform='pandas')

,transformers,"[('AgeImpute', ...), ('EmbarkedImpute', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'mean'
,fill_value,None


In [56]:
colTrans1 = DropAndImpute.fit_transform(X_train)
colTrans1

,AgeImpute__Age,EmbarkedImpute__Embarked,CabinImpute__Cabin,CabinImpute__missingindicator_Cabin,remainder__PassengerId,remainder__Pclass,remainder__Name,remainder__Sex,remainder__SibSp,remainder__Parch,remainder__Ticket,remainder__Fare
140,29.745184,C,0,True,141,3,"Boulos, Mrs. Joseph (Sultana)",female,0,2,2678,15.2458
439,31.000000,S,0,True,440,2,"Kvillner, Mr. Johan Henrik Johannesson",male,0,0,C.A. 18723,10.5000
817,31.000000,C,0,True,818,2,"Mallet, Mr. Albert",male,1,1,S.C./PARIS 2079,37.0042
378,20.000000,C,0,True,379,3,"Betros, Mr. Tannous",male,0,0,2648,4.0125
491,21.000000,S,0,True,492,3,"Windelov, Mr. Einar",male,0,0,SOTON/OQ 3101317,7.2500
...,...,...,...,...,...,...,...,...,...,...,...,...
835,39.000000,C,E49,False,836,1,"Compton, Miss. Sara Rebecca",female,1,1,PC 17756,83.1583
192,19.000000,S,0,True,193,3,"Andersen-Jensen, Miss. Carla Christine Nielsine",female,1,0,350046,7.8542
629,29.745184,Q,0,True,630,3,"O'Connell, Mr. Patrick D",male,0,0,334912,7.7333
559,36.000000,S,0,True,560,3,"de Messemaeker, Mrs. Guillaume Joseph (Emma)",female,1,0,345572,17.4000


In [57]:
colTrans1.columns

Index(['AgeImpute__Age', 'EmbarkedImpute__Embarked', 'CabinImpute__Cabin',
       'CabinImpute__missingindicator_Cabin', 'remainder__PassengerId',
       'remainder__Pclass', 'remainder__Name', 'remainder__Sex',
       'remainder__SibSp', 'remainder__Parch', 'remainder__Ticket',
       'remainder__Fare'],
      dtype='object')

In [58]:
colTrans1["remainder__Name"].apply(lambda x: x.split(",")[1].split()[0])

140     Mrs.
439      Mr.
817      Mr.
378      Mr.
491      Mr.
       ...  
835    Miss.
192    Miss.
629      Mr.
559     Mrs.
684      Mr.
Name: remainder__Name, Length: 712, dtype: object

In [59]:
from pipelineHelpers import extractTitle, extractAgeCategory, famCategory, ticketCategory, cabinCategory

def get_title_name(self, input_features):
    return ['NameTitle']

titleTransformer = FunctionTransformer(
    extractTitle,
    feature_names_out=get_title_name
)

def get_Age_Category_name(self, input_features):
    return ['AgeCategory']

ageTransformer = FunctionTransformer(
    extractAgeCategory,
    feature_names_out=get_Age_Category_name
)

def get_family_category(self, input_features):
    return ['FamSize']

famNumToCategoryTransformer = FunctionTransformer(
    famCategory,
    feature_names_out=get_family_category
)

def get_ticket_category(self, input_features):
    return ['TicketCat']

ticketCategoryTransformer = FunctionTransformer(
    ticketCategory,
    feature_names_out=get_ticket_category
)

def get_cabin_category(self, input_features):
    return ['CabinCategory']

cabinCategoryTransformer = FunctionTransformer(
    cabinCategory,
    feature_names_out=get_cabin_category
)

In [60]:
extractTitle(colTrans1["remainder__Name"])

,NameTitle
140,Mrs.
439,Mr.
817,Mr.
378,Mr.
491,Mr.
...,...
835,Miss.
192,Miss.
629,Mr.
559,Mrs.


In [61]:
extractAgeCategory(colTrans1["AgeImpute__Age"])

,AgeCategory
140,Peak
439,Peak
817,Peak
378,Peak
491,Peak
...,...
835,Peak
192,teen
629,Peak
559,Peak


In [62]:
famCategory(colTrans1.iloc[:, 8:10])

,FamSize
140,Small_fam
439,Alone
817,Small_fam
378,Alone
491,Alone
...,...
835,Small_fam
192,Small_fam
629,Alone
559,Small_fam


In [63]:
ticketCategory(colTrans1["remainder__Ticket"])

,TicketCat
140,NA
439,C.A.
817,S.C./PARIS
378,NA
491,SOTON/OQ
...,...
835,PC
192,NA
629,NA
559,NA


In [64]:
ticketCategory(colTrans1["remainder__Ticket"]).info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 140 to 684
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   TicketCat  712 non-null    object
dtypes: object(1)
memory usage: 11.1+ KB


In [65]:
cabinCategory(colTrans1["CabinImpute__Cabin"])

,CabinCategory
140,NA
439,NA
817,NA
378,NA
491,NA
...,...
835,E
192,NA
629,NA
559,NA


In [66]:
cabinCategory(colTrans1["CabinImpute__Cabin"]).info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 140 to 684
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   CabinCategory  712 non-null    object
dtypes: object(1)
memory usage: 11.1+ KB


In [67]:
CategoriesConstruct = ColumnTransformer(
    transformers= [
        ("TitleExtract", titleTransformer, [6]),
        ("ExtractAgeCategory", ageTransformer, [0]),
        ("FamCategory", famNumToCategoryTransformer, [8,9]),
        ("TicketCategory", ticketCategoryTransformer, [10]),
        ("CabinCategory", cabinCategoryTransformer, [2]),
    ], remainder='passthrough'
)

In [68]:
CategoriesConstruct.set_output(transform='pandas')

,transformers,"[('TitleExtract', ...), ('ExtractAgeCategory', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,func,<function ext...0018986C91C60>
,inverse_func,None
,validate,False


In [69]:
colTrans2 = CategoriesConstruct.fit_transform(colTrans1)
colTrans2

,TitleExtract__NameTitle,ExtractAgeCategory__AgeCategory,FamCategory__FamSize,TicketCategory__TicketCat,CabinCategory__CabinCategory,remainder__EmbarkedImpute__Embarked,remainder__CabinImpute__missingindicator_Cabin,remainder__remainder__PassengerId,remainder__remainder__Pclass,remainder__remainder__Sex,remainder__remainder__Fare
140,Mrs.,Peak,Small_fam,NA,NA,C,True,141,3,female,15.2458
439,Mr.,Peak,Alone,C.A.,NA,S,True,440,2,male,10.5000
817,Mr.,Peak,Small_fam,S.C./PARIS,NA,C,True,818,2,male,37.0042
378,Mr.,Peak,Alone,NA,NA,C,True,379,3,male,4.0125
491,Mr.,Peak,Alone,SOTON/OQ,NA,S,True,492,3,male,7.2500
...,...,...,...,...,...,...,...,...,...,...,...
835,Miss.,Peak,Small_fam,PC,E,C,False,836,1,female,83.1583
192,Miss.,teen,Small_fam,NA,NA,S,True,193,3,female,7.8542
629,Mr.,Peak,Alone,NA,NA,Q,True,630,3,male,7.7333
559,Mrs.,Peak,Small_fam,NA,NA,S,True,560,3,female,17.4000


In [70]:
colTrans2.info()                                #Missing Indicator attribute contained bool datatype, dont use OHE there

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 140 to 684
Data columns (total 11 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   TitleExtract__NameTitle                         712 non-null    object 
 1   ExtractAgeCategory__AgeCategory                 712 non-null    object 
 2   FamCategory__FamSize                            712 non-null    object 
 3   TicketCategory__TicketCat                       712 non-null    object 
 4   CabinCategory__CabinCategory                    712 non-null    object 
 5   remainder__EmbarkedImpute__Embarked             712 non-null    object 
 6   remainder__CabinImpute__missingindicator_Cabin  712 non-null    object 
 7   remainder__remainder__PassengerId               712 non-null    int64  
 8   remainder__remainder__Pclass                    712 non-null    int64  
 9   remainder__remainder__Sex                     

In [71]:
categoricalEncoding = ColumnTransformer(
    transformers = [
        ("OHE", OneHotEncoder(sparse_output=False, dtype=np.int32, drop='first'), [0,1,2,4,5,6,8]),
        ("OHE_Ticket", OneHotEncoder(sparse_output=False, dtype=np.int32, drop='first', max_categories=10, handle_unknown='infrequent_if_exist'), [3])
    ], remainder='drop'
)

In [72]:
categoricalEncoding.set_output(transform='pandas')

,transformers,"[('OHE', ...), ('OHE_Ticket', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,categories,'auto'
,drop,'first'
,sparse_output,False


In [73]:
category = categoricalEncoding.fit_transform(colTrans2)
category

,OHE__TitleExtract__NameTitle_Col.,OHE__TitleExtract__NameTitle_Dr.,OHE__TitleExtract__NameTitle_Lady.,OHE__TitleExtract__NameTitle_Major.,OHE__TitleExtract__NameTitle_Master.,OHE__TitleExtract__NameTitle_Miss.,OHE__TitleExtract__NameTitle_Mlle.,OHE__TitleExtract__NameTitle_Mme.,OHE__TitleExtract__NameTitle_Mr.,OHE__TitleExtract__NameTitle_Mrs.,...,OHE__remainder__remainder__Pclass_3,OHE_Ticket__TicketCategory__TicketCat_A/5.,OHE_Ticket__TicketCategory__TicketCat_C.A.,OHE_Ticket__TicketCategory__TicketCat_CA.,OHE_Ticket__TicketCategory__TicketCat_NA,OHE_Ticket__TicketCategory__TicketCat_PC,OHE_Ticket__TicketCategory__TicketCat_SOTON/OQ,OHE_Ticket__TicketCategory__TicketCat_STON/O,OHE_Ticket__TicketCategory__TicketCat_W./C.,OHE_Ticket__TicketCategory__TicketCat_infrequent_sklearn
140,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,1,0,0,0,0,0
439,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
817,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
378,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,0
491,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
192,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
629,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,0
559,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,1,0,0,0,0,0


In [74]:
numericalScaling = ColumnTransformer(
    transformers=[
        ("transform", PowerTransformer(standardize=True, copy=False), [10])
    ], remainder='drop'
)

In [80]:
numericalScaling.set_output(transform='pandas')
numerical = numericalScaling.fit_transform(colTrans2)
numerical

,transform__remainder__remainder__Fare
140,-0.135470
439,-0.510038
817,0.735932
378,-1.461543
491,-0.881743
...,...
835,1.490594
192,-0.801672
629,-0.817206
559,-0.003418


In [81]:
AfterTransformPipe = Pipeline(
    [
        ("categoricalEncoding", categoricalEncoding), 
        ("numericalScaling", numericalScaling)
    ]
)

In [82]:
pipe = Pipeline(
    [
        ("DropAndImpute", DropAndImpute),
        ("FeatureConstruction", CategoriesConstruct),
        ("FeatureTransformation", AfterTransformPipe)
    ]
)

In [83]:
pipe.set_output(transform="pandas")

,steps,"[('DropAndImpute', ...), ('FeatureConstruction', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('AgeImpute', ...), ('EmbarkedImpute', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
